# NLP Filtro de Spam

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Creación Spark

In [1]:
import os, subprocess

java8_home = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home"

os.environ["JAVA_HOME"] = java8_home
os.environ["PATH"] = os.path.join(java8_home, "bin") + os.pathsep + os.environ.get("PATH","")

os.environ["HADOOP_USER_NAME"] = os.environ.get("USER", "tomas")

print("JAVA_HOME fijado a:", os.environ["JAVA_HOME"])
try:
    print("which java (kernel):", subprocess.check_output(["which","java"]).decode().strip())
    print("java -version (kernel):")
    print(subprocess.check_output(["java","-version"], stderr=subprocess.STDOUT).decode())
except Exception as e:
    print("Error llamando a java desde kernel:", e)

JAVA_HOME fijado a: /Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home
which java (kernel): /Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/bin/java
java -version (kernel):
openjdk version "1.8.0_292"
OpenJDK Runtime Environment (AdoptOpenJDK)(build 1.8.0_292-b10)
OpenJDK 64-Bit Server VM (AdoptOpenJDK)(build 25.292-b10, mixed mode)



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

25/09/13 23:09:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/09/13 23:09:09 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


## Importación de Datos

In [4]:
data = spark.read.csv("../PySparkCourse/MLData/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Preparación de los Datos

In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length',length(data['text']))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Transformación de los Datos

- **Tokenizer:** Tokeniza el texto en palabras simples
- **SotpWordsRemover:** Genera una lista por defector para inglés, se le puede para una StopWords personalizada
- **CountVectorizer:** Aprende un vocabulario y transforma tokens a un vector de frecuencias
- **IDF:** Combina frecuencia por documento con la rareza clobal del término (Vector)
- **StringIndexer:** Convierte la etiqueta de texto a numérica

In [11]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vector

In [12]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [13]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

## Pipeline

In [21]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[
    ham_spam_to_num,
    tokenizer,
    stopremove,
    count_vec,
    idf,
    clean_up
])

In [19]:
cleaner = pipeline.fit(data)

In [20]:
clean_data = cleaner.transform(data)

## Conjunto de Entrenamiento

In [24]:
clean_data = clean_data.select(['label','features'])

In [25]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [26]:
training, testing = clean_data.randomSplit([0.7,0.3])

## Modelo

In [27]:
from pyspark.ml.classification import NaiveBayes

In [28]:
nb = NaiveBayes()
spam_predictor = nb.fit(training)

25/09/14 00:20:11 WARN DAGScheduler: Broadcasting large task binary with size 1132.6 KiB
25/09/14 00:20:14 WARN DAGScheduler: Broadcasting large task binary with size 1115.8 KiB


In [29]:
test_results = spam_predictor.transform(testing)

## Evaluación

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print(f"Accuracy of model at predicting spam was: {acc}")

25/09/14 00:21:16 WARN DAGScheduler: Broadcasting large task binary with size 1354.8 KiB


Accuracy of model at predicting spam was: 0.9228959442154869
